In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
hotel=pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')

In [ ]:
hotel.head()

In [ ]:
hotel.tail()

In [ ]:
hotel.info()

In [ ]:
# checking for null values
hotel.isnull().sum()

In [ ]:
# dropping company column as it contains many null values
hotel.drop('company',inplace=True,axis=1)

In [ ]:
hotel.shape

In [ ]:
# filling children and agents columns with median values
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='median')
imputer.fit(hotel[['children','agent']])
hotel[['children','agent']]=imputer.transform(hotel[['children','agent']])

In [ ]:
hotel.isnull().sum()

In [ ]:
hotel['country'].head(5)

In [ ]:
hotel['country'].fillna(str(hotel['country'].mode().values[0]),inplace=True)

In [ ]:
hotel.info()

In [ ]:
hotel.describe()

In [ ]:
hotel.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

## **Which hotel has more number of bookings?**

In [ ]:
plt.figure(figsize=(10,5))
data = hotel.groupby("hotel")["hotel"].count()
colors = ['green','blue']
data.plot.pie(autopct="%.1f%%",colors=colors);
centre_circle = plt.Circle((0,0),0.70,fc='black')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.tight_layout()

### > City Hotel has more number of bookings. 

## **In which year does City Hotel had most bookings?**

In [ ]:
data1=hotel[hotel['hotel']=='City Hotel']
data1.head()

In [ ]:
plt.figure(figsize=(12,5))
sns.countplot(y='arrival_date_year',data=data1,color='grey')

### > 2016 was the year when City Hotel had most bookings. 

## **Which month was busiest for City Hotel in 2016?**

In [ ]:
data2=data1[data1['arrival_date_year']==2016]
data2.head()

In [ ]:
df=data2['arrival_date_month'].value_counts().reset_index().rename(columns={'Month':'count','index':'Month'})
df['mon']=df['Month']
number={'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
df['mon']=df['mon'].map(number)
df.sort_values(by='mon',inplace=True)
df.drop('mon',axis=1,inplace=True)
df

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=30)
sns.pointplot(x='Month',y='arrival_date_month',data=df,color='yellow')

### > October month had most number of guests in City Hotel during 2016. 

## **Monthwise booking count of both the hotels**

In [ ]:
df1=hotel[['hotel','arrival_date_month']]
df1.head()

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=30)
sns.countplot(x='arrival_date_month',data=df1,hue='hotel',palette='magma')

### > City Hotel has highest number of bookings in August and lowest in January.
### > Resort Hotel has highest number of bookings in August and lowest in January.

## **Which hotel has most cancellations?**

In [ ]:
dff=hotel[hotel['is_canceled']==0]
df2=dff.groupby('hotel')['is_canceled'].count()
df2

### > City Hotel had most cancellations. 

## **Which hotel has more number of repeated guests?**

In [ ]:
dff1=hotel[hotel['is_repeated_guest']==1]
plt.figure(figsize=(10,5))
data = dff1.groupby("hotel")["is_repeated_guest"].count()
colors = ['orange','blue']
explode=(0,0.1)
data.plot.pie(autopct="%.1f%%",colors=colors,explode=explode);

plt.tight_layout()

### > City hotel has more number of repeated guests. 

## **Countrywise total number of guests**

In [ ]:
guests = hotel[hotel['is_canceled'] == 0]['country'].value_counts().reset_index()
guests.columns = ['country', 'No of guests']
guests

## **Which room is more costly?**

In [ ]:
plt.figure(figsize=(12,6))
ds=hotel[hotel['is_canceled']==0]
sns.boxplot(x = 'reserved_room_type', y = 'adr',data=ds,hue='hotel',palette='Set1')